In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import jaxopt
import jax.scipy as jscipy
import scipy
from functools import partial
import api

from jax.config import config; config.update("jax_enable_x64", True)

In [2]:
N_omega = 401
omega = jnp.linspace(-30, 30, N_omega)
extended_omega = jnp.linspace(omega[0], omega[-1], 2*N_omega - 1)
delta_omega = np.abs(omega[1] - omega[0])
z = 1.#2*10**-3
delta_k = 1.j*np.diag(omega)
n = .5

In [10]:
# Initialize the pump 
beta = 0.1*(jax.random.uniform(key=jax.random.PRNGKey(np.random.randint(999)), shape=(2*N_omega - 1, )) - 1)

In [20]:
jax.devices()

[CpuDevice(id=0)]

In [11]:
# Create the minimization solver
optimizer = jaxopt.ScipyMinimize(method="BFGS", fun=api.problem, maxiter=1, jit = False)

In [13]:
# Implement penalty method
import time
k = 0
pumps = []
for i in range(20):
    tic = time.time()
    results_n = []
    results_purity = []
    for j in range(20):
        beta, _ = optimizer.run(beta, omega, z, n, k)
        results = api.get_observables(beta, omega, z)
        results_n.append(results[0])
        results_purity.append(results[1])
        print(j)
    k = 3**i
    print("Observables: ", results)
    print("Penalty index: ", i)
    print("Time for a  loop: ", time.time() - tic)
    pumps.append(beta.copy())

KeyboardInterrupt: 